In [1]:
import matplotlib
import seaborn
%matplotlib inline

import numpy as np
import pandas as pd

import keras
import nltk
import gensim
from gensim.models import word2vec, doc2vec

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

Using Theano backend.
Using cuDNN version 5110 on context None
Mapped name None to device cuda: GeForce GTX 1080 (0000:01:00.0)


In [2]:
train = pd.read_csv('../data/spacy_train.csv', index_col='Id')
test = pd.read_csv('../data/spacy_test.csv', index_col='Id')

In [3]:
train.shape, test.shape

((102544, 7), (34194, 1))

# CountVectorizer

In [15]:
vectorizer = CountVectorizer(analyzer='word',
                             ngram_range=(1, 3),
                             min_df=20,
#                              max_df=0.7,
#                              max_features=40000,
#                              stop_words='english',
                             lowercase=False)

vectorizer.fit(pd.concat([train.Text, test.Text]))

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=20,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [16]:
train_features = vectorizer.transform(train.Text)
test_features = vectorizer.transform(test.Text)

In [17]:
X_train, X_valid, y_train, y_valid = train_test_split(train_features, train.Sentiment.values, test_size=0.3)
X_test = test_features

In [18]:
sgd_est = SGDClassifier(loss='modified_huber', penalty='l1', n_iter=5, n_jobs=10)
sgd_est.fit(X_train, y_train)
print('Train: ', sgd_est.score(X_train, y_train))
print('Valid: ', sgd_est.score(X_valid, y_valid))

Train:  0.681652270828
Valid:  0.52785723573


In [14]:
pred_test = sgd_est.predict(X_test)
prediction = pd.DataFrame(data={'Id': test.Id, 'Sentiment': pred_test}, index=test.index)
prediction.to_csv('data/prediction.csv', index=False)
print(prediction.head())

   Id  Sentiment
0   0          3
1   1          3
2   2          3
3   3          5
4   4          5
